In [ ]:

import torch
import torch.nn as nn
import torch.nn.functional as F
import cv2
import numpy as np
import albumentations as A
from albumentations.pytorch import ToTensorV2
import os


# 1) 학습 완료된 모델 파일의 경로
MODEL_PATH = "crnn_recognizer_final.pth"

# 2) 테스트하고 싶은 이미지 한 장의 경로
#    (인식 모델이므로, 글자 부분만 잘려진 이미지를 넣어야 합니다.)
IMAGE_PATH = "test03.jpg" # <-- 테스트할 이미지 경로로 수정!

# 3) 학습에 사용했던 gt.txt 파일의 경로 (CHARACTER_SET 자동 생성을 위해 필요)
GT_FILE_FOR_CHARSET = r"C:\Users\User\DBNet_OCR\data\crop\gt.txt" 

# 4) 실행 장치 설정
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"



def generate_character_set(gt_file):
    """gt.txt 파일로부터 CHARACTER_SET을 자동으로 생성합니다."""
    if not os.path.exists(gt_file):
        print(f"오류: gt.txt 파일을 찾을 수 없습니다! 경로를 확인하세요: {gt_file}")
        return None
    all_characters = set()
    with open(gt_file, 'r', encoding='utf-8') as f:
        for line in f:
            try:
                _, text = line.strip().split('\t', 1)
                for char in text:
                    all_characters.add(char)
            except ValueError:
                continue
    sorted_characters = sorted(list(all_characters))
    final_charset = "".join(sorted_characters)
    return final_charset

class CTCLabelConverter:
    """텍스트와 숫자 인덱스 간 변환기"""
    def __init__(self, character_set):
        self.character_set = ["-"] + list(character_set)
        self.char_to_idx = {char: i for i, char in enumerate(self.character_set)}
        self.idx_to_char = {i: char for i, char in enumerate(self.character_set)}
    
    def decode(self, indices):
        text = []
        last_idx = 0
        for idx in indices:
            idx_item = idx.item()
            if idx_item == 0: last_idx = 0; continue
            if idx_item == last_idx: continue
            text.append(self.idx_to_char[idx_item])
            last_idx = idx_item
        return "".join(text)

    def get_num_classes(self):
        return len(self.character_set)

def get_recognition_transforms(height, width):
    """테스트용 이미지 변환기"""
    return A.Compose([
        A.Resize(height, width),
        A.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
        ToTensorV2(),
    ])

class CRNN(nn.Module):
    """CRNN 모델 아키텍처 (학습 때와 동일)"""
    def __init__(self, num_chars, rnn_hidden_size=256, rnn_layers=2):
        super().__init__()
        self.cnn = nn.Sequential(
            nn.Conv2d(3, 64, 3, 1, 1), nn.ReLU(True), nn.MaxPool2d(2, 2),
            nn.Conv2d(64, 128, 3, 1, 1), nn.ReLU(True), nn.MaxPool2d(2, 2),
            nn.Conv2d(128, 256, 3, 1, 1), nn.BatchNorm2d(256), nn.ReLU(True),
            nn.Conv2d(256, 256, 3, 1, 1), nn.ReLU(True), nn.MaxPool2d((2, 1), (2, 1)),
            nn.Conv2d(256, 512, 3, 1, 1), nn.BatchNorm2d(512), nn.ReLU(True),
            nn.Conv2d(512, 512, 3, 1, 1), nn.ReLU(True), nn.MaxPool2d((2, 1), (2, 1)),
            nn.Conv2d(512, 512, (2,1), 1, 0), nn.BatchNorm2d(512), nn.ReLU(True)
        )
        self.rnn = nn.LSTM(input_size=512, hidden_size=rnn_hidden_size, num_layers=rnn_layers, bidirectional=True, dropout=0.5)
        self.classifier = nn.Linear(rnn_hidden_size * 2, num_chars)
    def forward(self, x):
        features = self.cnn(x); b, c, h, w = features.size()
        assert h == 1, "CNN 출력의 높이는 1이어야 합니다."
        features = features.squeeze(2).permute(2, 0, 1); rnn_output, _ = self.rnn(features)
        return self.classifier(rnn_output)

class Recognizer:
    """학습된 모델로 추론을 수행하는 클래스"""
    def __init__(self, model_path, converter, device, img_height=32, img_width=100):
        self.device = device; self.converter = converter
        num_classes = self.converter.get_num_classes()
        self.model = CRNN(num_chars=num_classes).to(self.device)
        self.model.load_state_dict(torch.load(model_path, map_location=self.device))
        self.model.eval()
        self.transform = get_recognition_transforms(img_height, img_width)

    def predict(self, image_path):
        try:
            image = cv2.imread(image_path)
            if image is None: return f"이미지 파일을 찾거나 열 수 없습니다: {image_path}"
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        except Exception as e:
            return f"이미지 로드 오류: {e}"
        image_tensor = self.transform(image=image)['image'].unsqueeze(0).to(self.device)
        with torch.no_grad():
            preds = self.model(image_tensor)
            preds_idx = preds.argmax(2).permute(1, 0)
        decoded_text = self.converter.decode(preds_idx[0])
        return decoded_text

# ====================================================================================
# 메인 실행 블록
# ====================================================================================
if __name__ == '__main__':
    print("===== 텍스트 인식(Recognition) 테스트 시작 =====")
    print(f"사용 장치: {DEVICE}")

    # 1. 학습에 사용된 글자셋(CHARACTER_SET) 자동 생성
    print(f"\n>> 학습 데이터({os.path.basename(GT_FILE_FOR_CHARSET)})에서 문자셋을 생성합니다...")
    charset = generate_character_set(GT_FILE_FOR_CHARSET)
    
    if charset:
        # 2. 추론기(Recognizer) 초기화
        try:
            print(f"\n>> '{MODEL_PATH}' 모델을 로드합니다...")
            recognizer = Recognizer(MODEL_PATH, CTCLabelConverter(charset), DEVICE)
            print("모델 로드 완료.")
            
            # 3. 이미지 예측 실행
            print(f"\n>> '{os.path.basename(IMAGE_PATH)}' 이미지의 텍스트를 예측합니다...")
            predicted_text = recognizer.predict(IMAGE_PATH)
            
            # 4. 최종 결과 출력
            print("\n" + "="*20 + " 최종 결과 " + "="*20)
            print(f"입력 이미지: {IMAGE_PATH}")
            print(f"모델 예측 텍스트: '{predicted_text}'")
            print("="*53)

        except FileNotFoundError:
            print(f"\n[오류] 모델 파일 또는 이미지 파일을 찾을 수 없습니다. 상단의 경로 설정을 확인해주세요.")
        except Exception as e:
            print(f"\n예상치 못한 오류가 발생했습니다: {e}")
    else:
        print("\n[오류] 문자셋 생성에 실패하여 테스트를 진행할 수 없습니다.")

===== 텍스트 인식(Recognition) 테스트 시작 =====
사용 장치: cuda

>> 학습 데이터(gt.txt)에서 문자셋을 생성합니다...

>> 'crnn_recognizer_final.pth' 모델을 로드합니다...
모델 로드 완료.

>> 'test03.jpg' 이미지의 텍스트를 예측합니다...

==================== 최종 결과 ====================
입력 이미지: test03.jpg
모델 예측 텍스트: '실'


C:\Users\User\AppData\Local\Temp\ipykernel_21940\2571165164.py:108: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.model.load_state_dict(torch.load(model_path, map_locat